In [ ]:
using LilGuys
using CairoMakie


In [ ]:
using PythonCall
agama = pyimport("agama")
np = pyimport("numpy")

In [ ]:
halo = agama.Potential(type="NFW", scaleRadius=2.7, mass=1.0)

In [ ]:
using LinearAlgebra: norm, det

In [ ]:
a = [0.3, 0.9, -0.2]
a /= norm(a)

In [ ]:
N = 100
r = LinRange(-2, 2, N)

x = 10 .^ r  .*   LilGuys.rand_unit(1)'

x_np = np.array(x)

In [ ]:
hess = pyconvert(Matrix{Float64}, halo.eval(x_np, der=true))'

In [ ]:
hess_labels = [
    "xx", "yy", "zz", 
    "xy", "yz", "zx"
    ]

In [ ]:
for i in 1:10
    A = hess[:, i]
    M = [A[1] A[5] A[6]
        A[5] A[2]  A[4]
        A[6] A[4] A[3]
        ]

    println(A[1] + A[2] + A[3])
end

In [ ]:
tr = @. -(hess[1, :] + hess[2, :] + hess[3, :])

In [ ]:
fig = Figure()


filt = r .> -1
ax = Axis(fig[1,1])
for i in 1:6
    lines!(r[filt], (hess[i, filt]) ./ maximum(abs.(hess[i, filt])))
end

fig

In [ ]:
fig = Figure()


filt = r .> -1
for i in 1:6
    ax = Axis(fig[(i-1)%2 + 1, (i-1)÷2 + 1],
        ylabel=hess_labels[i],
        xlabel = "log r",
    )

    lines!(r[filt], (hess[i, filt]) .* (10 .^ r)[filt] .^ (2))
end

fig

In [ ]:
h = NFW(r_s=2.7, M200=1)

In [ ]:
f = lines(r, log10.(-tr))

lines!(r, log10.(calc_ρ.(h, 10 .^ r)) .+ log10(8pi))
f